# Generate Segmentation Images based on NetSurfaceProblem

In [1]:
%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian

from tifffile import imread, imsave
import _pickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from Netsurface3d import NetSurf3d
from data4d import Data4d
from stitch_surfaces import StitchSurfaces
from netsurface3d_radial import NetSurf3dRadial
from data4d_radial import Data4dRadial
from Visualization import Visualization
from CrossSection import CrossSection

# Load and Initialize

In [2]:
#filenames of images and of mask have to be in same order

#filenames = [ '/Users/wissmann/BobSeg/FlySegCropped.tif']
#filenames = [ '/Users/wissmann/BobSeg/Cell_croppedModel-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1)-1.tif']
filenames = [ '/Users/wissmann/BobSeg/Vecad-flidsRed-48hpf.lif - fish1-48hpf-1.tif']
filenames_mask = np.array([['/Users/wissmann/BobSeg/fish_mask.tif',None,None]])
#filenames = [ '/Users/wissmann/BobSeg/easy2surfaces.tif']
#filenames_mask = [ '/Users/wissmann/BobSeg/easymask.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack3.tif']
#filenames = [ '/Users/wissmann/BobSeg/debugdx.tif']
#filenames = ['/Users/wissmann/BobSeg/nonquadratictest.tif']

In [3]:
#Methods : [' z  ,  y ,  x , rad']
chosen  =  [  1  ,  0 ,  0 ,  1  ]

#if you change chosen, Kernel restart is required

In [4]:
# Parameters for Netsurf with Base Graph
#             [ z, y, x]
K           = [50,40,30]                 
max_delta_k = [2 , 2, 2]
dx          = [10,10, 4]
dy          = [10, 4,10]
surfaces    = [ 2, 2, 2]
min_dist    = [ 2, 8, 2]                 #In terms of K
max_dist    = [50,50,50]                 #In terms of K
plot_base_graph =False                   

In [5]:
# Parameters for Radial Netsurf
K_Rad            = 40
max_delta_k_rad  = 3
max_rs           = (800,740,220)                                   #In px (x,y,z)
min_rs           = (1,1,1)                                         #In px (x,y,z)
centers          = [(500,500,100),(200,200,100)]                   #In px (x,y,z)

# Segmentation

In [6]:
data={}
Bg={}
Ra={}

#Base graph
for i in range(3):
    if chosen[i]==1:
        data[i]=Data4d(filenames, axis=i, filenames_mask=filenames_mask[:,i], pixelsize=(1.,1.,1.), silent=False, plane=None )
        data[i].set_seg_params(K[i],max_delta_k[i],dx[i],dy[i],surfaces[i],min_dist[i],max_dist[i])
        Bg[i] = data[i].init_object("Bg_%s"%i)
        data[i].add_object_at( Bg[i], 0, data[i].images[0].shape[0], frame=0, frame_to=0, segment_it=True, plot_base_graph=plot_base_graph )
    else: data[i]=None
        
#Radial
if chosen[3]==1:
    data[3]=Data4dRadial( filenames, pixelsize=(1.,1.,1.), silent=False )
    data[3].set_seg_params(K_Rad,max_delta_k_rad)
    for center in range(len(centers)):
        Ra[center]=data[3].init_object("Ra_%s"%center)
        data[3].add_object_at(Ra[center], min_rs, max_rs, frame=0, seed=centers[center], frame_to=0, segment_it=True )
else: data[3]=None
    
data[5]=Data4d(filenames, axis=0, filenames_mask=filenames_mask[:,0], pixelsize=(1.,1.,1.), silent=True, plane=None )
images=data[5].images

Dimensions (z,y,x) of frame 0 :  (213, 738, 792)
Searching for surface(s) along z direction
Dimensions (y,x) of mask 0 :  (738, 792)
Added appearance for "Bg_0" in frame 0
Number of columns: 3038
Number of Surfaces: 2
Done with surface 0
Elapsed time is 7.350883722305298 seconds.
Done with surface 1
Elapsed time is 7.875776052474976 seconds.
      Optimum energy:  202317.0
      Nodes in/out:  203428 100372
Dimensions of frame 0 :  (213, 738, 792)
Added appearance for "Ra_0" in frame 0 with seed coordinates [500. 500. 100.]
      Optimum energy:  8469.0
      Nodes in/out:  8799 11681
Added appearance for "Ra_1" in frame 0 with seed coordinates [200. 200. 100.]
      Optimum energy:  11880.0
      Nodes in/out:  4588 15892


# Visualization

    Colour code for both 3d visualization and 2d cross-section plots: 
        Base graph (x,y,z): (green,blue,gold)
        Radial            : red

In [7]:
vis = Visualization(data, images)

    show_single: shows all meshes from all chosen netsurf possibilities
    show_union: shows one union mesh of the single meshes
    export: exports meshes as .obj files e.g. to be opened in blender
    stitch: in case of base graph with more than 1 surface, stitches the surfaces together to simulate volume
        setting stitch to True applies for all base graph netsurfs

In [8]:
vis.show_frame(f=0, show_union=False, show_single=True, 
                   stackUnits=[1.,1.,1.], raise_window=True, export=False, stitch=False)

/Users/wissmann/py37/lib/python3.7/site-packages/pyopencl/cffi_cl.py:1521: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


vertices (3038, 3)
vertices (2, 3038, 3)
m <spimagine.gui.mesh.Mesh object at 0x15cf23b00>
m <spimagine.gui.mesh.Mesh object at 0x15cf23c88>


In [9]:
z_extents = [180,181]
z_levels  = np.arange(z_extents[0],z_extents[1]+1, step=1)
vis.show_sections(f=0,plane_orig=[0,0,0], plane_normal=[0,1,0], num_slices=z_levels,show_image=True) #in [x,y,z]

(1, 2, 3038, 3)
in process (2, 40, 2, 3)
(80, 2, 3)
(80, 2, 2)
in process (2,)
(118, 2, 3)
(118, 2, 2)
in process (2, 79, 2, 3)
(158, 2, 3)
(158, 2, 2)
in process (2,)
(111, 2, 3)
(111, 2, 2)
